In [ ]:
import astropy as ap
from astropy import coordinates
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('test/one_slice/SpecObjPhot.csv')

In [ ]:
df.columns

In [ ]:
print(len(df))

In [ ]:
high_q = df[df['NQ'] > 2]

In [ ]:
print(len(high_q))

In [ ]:
gf = {}
# gf['G02'] = high_q.loc[(high_q['RA'] > 30.2-1) & (high_q['RA'] < 38.8+1)]
gf['G09'] = high_q.loc[(high_q['RA'] > 129.0-1) & (high_q['RA'] < 141.0+1)]
gf['G12'] = high_q.loc[(high_q['RA'] https://www.astro.ljmu.ac.uk/~ikb/research/gama_fields/> 174.0-1) & (high_q['RA'] < 186.0+1)]
gf['G15'] = high_q.loc[(high_q['RA'] > 211.5-1) & (high_q['RA'] < 223.5+1)]

In [ ]:
i = 0
for key in gf.keys():
    print(len(gf[key]))
    i += len(gf[key])
print(i)

In [ ]:
small_piece = gf['G09']
print(small_piece.iloc[99])

In [ ]:
coords1 = ap.coordinates.SkyCoord(small_piece['RA'], small_piece['DEC'], unit='deg')

In [ ]:
# small_piece = df.sample(200)
# coords2 = ap.coordinates.SkyCoord(small_piece['RA'], small_piece['DEC'], unit='deg')

In [ ]:
plt.scatter(coords1.ra, coords1.dec)
# plt.scatter(coords2.ra, coords2.dec)

In [ ]:
maxang = ap.units.Quantity(2.5, 'deg')

In [ ]:
# warning: slow!!!
output = ap.coordinates.search_around_sky(coords1, coords1, maxang)

In [ ]:
for o in output:
    print((len(o), o))

In [ ]:
plt.hist(output[2].value, bins=100)

In [ ]:
plt.hist(output[0], bins=len(np.unique(output[0])))

In [ ]:
np.array([output[0], output[1]]).T

In [ ]:
nps = 10 #mp.cpu_count()
pool = mp.Pool(nps - 1)

In [ ]:
coords1